In [1]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
mapa = pd.read_excel('mapa.xlsx', sheet_name='custos')
mapa = mapa.iloc[0:16, 0:16]
mapa = mapa.values.tolist()

---
## Criando o modelo

In [3]:
def create_data_model():
    data = {}
    data['matriz_distancia'] = mapa
    data['veiculos'] = 1 # um PCV possui apenas um veículo
    data['destino'] = 0 # saindo e voltando para o nó 0
    return data

--- 
### Modelo de rotas

In [4]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['matriz_distancia']),
                                       data['veiculos'], data['destino']) # convertendo os indices para nós
routing = pywrapcp.RoutingModel(manager) 

---
## Callback de distância
Formaliza as distâncias entre os nós

In [5]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['matriz_distancia'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

---
## Custos dos arcos
Ensina o solver a calcular os custos entre os nós

In [6]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

---
## Definindo parâmetros

In [7]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.seconds = 30
search_parameters.log_search = True

## Definindo o output do solver

In [8]:
def print_solution(manager, routing, solution):
    print('Objetivo: R$ {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Rota :\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Distância da rota: {} kilometros\n'.format(route_distance)

In [9]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objetivo: R$ 4421
Rota :
 0 -> 11 -> 15 -> 5 -> 13 -> 2 -> 7 -> 4 -> 3 -> 14 -> 9 -> 6 -> 1 -> 12 -> 10 -> 8 -> 0

